Get tweets and topics keywords embeddings, compute cosine similarity and get topic with highest score per tweet

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir("/content/drive/My Drive/IRProject")
os.listdir()

Mounted at /content/drive


['tweet_TR.json',
 'preprocessed_tweets.pickle',
 'tweets_emoSent2.json',
 'TopicModeling (1).ipynb',
 'topicsModel',
 'tweets_emoSentTop.json',
 'tweets_emoSentTopic.json',
 'tweets_emoSentTopics.json',
 'tweets_emoSentTopics2.json',
 'tweets_8Topics.json',
 'words_per_topic.pickle',
 'words_per_topic.json',
 'corpus.pkl',
 'dictionary.gensim',
 'lda.html',
 'tweets_no_dups.json',
 'preprocessed_tweets_no_dups.pickle',
 'model30.gensim.state',
 'model30.gensim',
 'model30.gensim.id2word',
 'model30.gensim.expElogbeta.npy',
 'TopicModeling.ipynb',
 'model10.gensim.state',
 'model10.gensim.id2word',
 'model10.gensim.expElogbeta.npy',
 'model10.gensim',
 'lda_10Topics.html',
 'TopicsPerTweet.csv',
 'tweets_topics_LDA.json',
 'wordCloud.png',
 'wordCloud2.png',
 'wordCloud.ipynb',
 'mallet-2.0.8',
 'dictionary_no_dups.gensim',
 'corpus_no_dups.pkl',
 'Untitled',
 'TopicModelingLDAMallet.ipynb',
 'tweets_keywords.pickle',
 'preprocessed_all_TR_50638.pickle',
 'corpus_50638.pkl',
 'dictiona

In [ ]:
# Load gensim models
import gensim.downloader as api
gl = api.load('glove-twitter-100')
#model = api.load("glove-wiki-gigaword-100")

[==================================================] 100.0% 387.1/387.1MB downloaded


In [ ]:
print(list(api.info()['models'].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [ ]:
# For each tweet / topic, get its keywords embeddings 
# keywords is a list of keywords for tweets or topics
def getEmbeddings(keywords):
  embeddings = []
  for line in keywords:
    emb = []
    for k in line:
      try:
        e = gl[k]
        emb.append(e)
      except:
        emb.append('NA')
    embeddings.append(emb)
  return embeddings


In [ ]:
# For each tweet / topic, compute mean of keywords embeddings
import numpy as np

def getMeanEmbeddings(embeddings):
  mean_embeddings = []
  for embs in embeddings:
    words = [e for e in embs if e != 'NA']
    if len(words) >= 1:
      avg = np.mean(words, axis=0)
      #if embs != 'NA':
      #avg = np.mean(words, axis=0)
      #avg = 0
      #for w in words:
        #avg += w
      #avg = avg / len(words)
      mean_embeddings.append(avg)
    else:
      mean_embeddings.append('NA')

  return mean_embeddings

In [ ]:
#from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# For each tweet mean embedding, compute its cosine similarity with each topic mean embedding and return topic with highest score
def getMostSimilarTopic(topic_mean_embeddings,tweets_mean_embeddings):
  topics = []
  for t in tweets_mean_embeddings:
    # Compute cosine similarity w/ each topic
    if t != 'NA':
      similarities = []
      for i in topic_mean_embeddings:
        sim = np.dot(t,i) / (np.linalg.norm(t) * np.linalg.norm(i))
        similarities.append(sim)
      print(similarities)  
      highest = max(similarities)
      pair = (similarities.index(highest),highest)
      topics.append(pair)
    else:
      topics.append('NA')
  return topics


In [ ]:
# Load files
import json
import pickle

with open ('topicKeywords.json', 'rb') as f:
    keywords = json.load(f)
with open ('tweets_keywords_50638.pickle', 'rb') as f:
    tweets_keywords = pickle.load(f)

In [ ]:
import re
topics_keywords = []
for k in keywords.keys():
  key = keywords[k]
  text = re.sub('[^a-z]+',' ',key) # remove everythibng that's not a word
  text = re.sub('^\s+','',text) # remove space(s) at start
  text = re.sub('\s+$','',text) # remove space(s) at end
  text = re.split('\s',text) # split words
  topics_keywords.append(text)

In [ ]:
len(tweets_keywords)

50638

In [ ]:
# Get embeddings
topics_embeddings = getEmbeddings(topics_keywords)
tweets_embeddings = getEmbeddings(tweets_keywords)

In [ ]:
# Get mean embeddings
topics_mean_embeddings = getMeanEmbeddings(topics_embeddings)
tweets_mean_embeddings = getMeanEmbeddings(tweets_embeddings)

In [ ]:
len(tweets_mean_embeddings)

50638

In [ ]:
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Compute cosine similarity betweet tweets and topics
tweets_topics = getMostSimilarTopic(topics_mean_embeddings, tweets_mean_embeddings)

In [ ]:
tweets_topics[0]

(4, 0.62428457)

In [ ]:
count = 0
for h in tweets_topics:
  if h == 'NA':
    count += 1
count

6182

In [ ]:
with open ('topicNames.json', 'rb') as f:
    topicNames = json.load(f)

In [ ]:
topicNames

{'0': '1_Corona_Masks_Government',
 '1': '2_Immunity_Positive_South_Africa',
 '2': '3_Hospital_Health',
 '3': '4_Coronavirus',
 '4': '5_India_Covid_Cases',
 '5': '6_Omicron_Variant',
 '6': '7_Symptoms_Lockdown',
 '7': '8_Travel_Quarantine',
 '8': '9_Social_Distance',
 '9': '10_Covid_Vaccination'}

In [ ]:
list_of_topics = []
for d in tweets_topics:
  if d != 'NA':
    list_of_topics.append(d[0])
  else:
    list_of_topics.append('NA')
len(list_of_topics)

50638

In [ ]:
topic_names = []
for l in list_of_topics:
  if l == 'NA':
    topic_names.append(l)
  else:
    for x in topicNames.keys():
      if str(l) == x:
        topic_names.append(topicNames[x])

In [ ]:
# Update json file with topics
from tqdm import tqdm
def updateJsonFile():
    jsonFile = open("all_TR_50638.json", "r") 
    data = json.load(jsonFile) 
    jsonFile.close()

    i = 0
    for tweet in tqdm(data): 
      tweet['topic'] = topic_names[i]
      i += 1

    with open("all_TR_50638_Topics.json", "w+") as jsonFile: 
        jsonFile.write(json.dumps(data))

In [ ]:
updateJsonFile()

100%|██████████| 50638/50638 [00:00<00:00, 1207289.32it/s]
